In [11]:
from pyignite.utils import __hashcode_fallback
from pyignite._cutils import hashcode
import inspect

print(inspect.getsource(__hashcode_fallback))

def __hashcode_fallback(data: Union[str, bytes, bytearray, memoryview]) -> int:
    if data is None:
        return 0

    if isinstance(data, str):
        """
        For strings we iterate over code point which are of the int type
        and can take up to 4 bytes and can only be positive.
        """
        result = 0
        for char in data:
            try:
                char_val = ord(char)
                result = int_overflow(31 * result + char_val)
            except TypeError:
                pass
    else:
        """
        For byte array we iterate over bytes which only take 1 byte. But
        according to protocol, bytes during hashing should be treated as signed
        integer numbers 8 bits long. On other hand elements in Python's `bytes`
        are unsigned. For this reason we use ctypes.c_byte() to make them
        signed.
        """
        result = 1
        for byte in data:
            byte = ctypes.c_byte(byte).value
            result = int_overflow(

In [13]:
import secrets

data = secrets.token_bytes(1024*1024)

In [14]:
%timeit hashcode(data)

869 µs ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit __hashcode_fallback(data)

464 ms ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
string = secrets.token_hex(64)

In [8]:
%timeit hashcode(string)

196 ns ± 2.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%timeit __hashcode_fallback(string)

35.8 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
